In [1]:
%pip install bigframes

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "admin.json"

In [3]:
PROJECT_ID = "mlops-433612"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
from vertexai.resources.preview.feature_store import FeatureGroup
from vertexai.resources.preview.feature_store import utils as fs_utils

FEATURE_GROUP_ID = "test_feature_group"  # @param {type:"string"}
DATASET_ID = "featurestore_demo"
TABLE_ID = "cleaned_table"

BQ_TABLE_URI = f'{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}'

feature_groups = FeatureGroup.list()

# Find the feature group with the desired ID
fg = next((fg for fg in feature_groups if fg.name == FEATURE_GROUP_ID), None)

if fg:
    print(f"Feature group found: {fg}")
else:
    print(f"Feature group with ID {FEATURE_GROUP_ID} not found.")

2024-09-15 21:25:27.304630: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-15 21:25:27.308783: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-15 21:25:27.324076: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 21:25:27.343696: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 21:25:27.348659: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-15 21:25:27.362568: I tensorflow/core/platform/cpu_feature_gu

Feature group found: <vertexai.resources.preview.feature_store.feature_group.FeatureGroup object at 0x7f2d23601a00> 
resource name: projects/994601067814/locations/us-central1/featureGroups/test_feature_group


In [5]:
from vertexai.resources.preview.feature_store import Feature

feature_names = ["Name","Age","Sex","Fare","Pclass","SibSp","PassengerId","Survived","Parch","Ticket","Cabin","Embarked"]
features = []

for feature in feature_names:
   f: Feature = fg.get_feature(feature)
   features.append(f)

In [11]:
import bigframes
import pandas as pd
from vertexai.resources.preview.feature_store import offline_store

entity_df = pd.DataFrame(
  data={
    "entity_id": [f"id-{i}" for i in range(0, 182)],
    "timestamp": [pd.Timestamp("2024-09-15 13:24:22.862434 UTC") for i in range(0, 182)],
  },
)

response = offline_store.fetch_historical_feature_values(
  entity_df=entity_df,
  features=features,
)

/home/mirai/.pyenv/versions/3.12.2/lib/python3.12/site-packages/bigframes/core/blocks.py:126: NullIndexPreviewWarning: Creating object with Null Index. Null Index is a preview feature.
  warnings.warn(
/home/mirai/.pyenv/versions/3.12.2/lib/python3.12/site-packages/bigframes/core/blocks.py:126: NullIndexPreviewWarning: Creating object with Null Index. Null Index is a preview feature.
  warnings.warn(
/home/mirai/.pyenv/versions/3.12.2/lib/python3.12/site-packages/bigframes/core/blocks.py:126: NullIndexPreviewWarning: Creating object with Null Index. Null Index is a preview feature.
  warnings.warn(


,entity_id,Name,Age,Sex,Fare,Pclass,SibSp,timestamp
0,id-44,"Cherry, Miss. Gladys",30.0,female,86.5,1,0,2024-09-15 13:24:22.862434+00:00
1,id-40,"Minahan, Dr. William Edward",44.0,male,90.0,1,2,2024-09-15 13:24:22.862434+00:00
2,id-126,"Kimball, Mr. Edwin Nelson Jr",42.0,male,52.5542,1,1,2024-09-15 13:24:22.862434+00:00
3,id-23,"Futrelle, Mr. Jacques Heath",37.0,male,53.1,1,1,2024-09-15 13:24:22.862434+00:00
4,id-145,"Calderhead, Mr. Edward Pennington",42.0,male,26.2875,1,0,2024-09-15 13:24:22.862434+00:00


In [15]:
response.to_csv('results.csv', index=False)